# Imports

In [9]:
import numpy as np
import pandas as pd
import math
import sys, os, time, datetime
import importlib as imp
from imp import reload

import matplotlib.pyplot as plt
from matplotlib import rcParams

import csv
import winsound
import threading

import requests, re
from detectorConnection import Detectotron as det

# Detectors

In [12]:
url = 'http://det.phy.bris.ac.uk'
REGEX = '\"(.*)\\n\"'
MAXVOLTAGE= 2.25 # Volts
MAXCURRENT = 25  # micro Amps

def set_bias(system, channel, current_uA, verbose=False):
# Set bias current NEW PHOTONSPOT SYSTEMS
# Channel is either int (1-24) or channel string i.e. CTRL2_A1
    if not -MAXCURRENT <= current_uA <= MAXCURRENT:
        print ('Please set the voltage to an interval between -%s uA to +%s uA' % (MAXCURRENT, MAXCURRENT))
        return
    command = '/bias2.php?cmd=bias_set&system={:}&channel={:}&current={:}'.format(system, channel, current_uA)
    requests.post(url+command)
    if verbose:
        print('Bias current on system {:} channel {:} set to {:.3f} uA'.format(system, channel, get_bias(system, channel)))

def delatch(channel,system = 'S'):
# Delatch the channel
    command = '/bias2.php?cmd=delatch&system={:}&channel={:}'.format(system, channel)
    requests.post(url+command)

def delatch_all(channels, system = 'S'):
    for ch in channels:
        delatch(ch,system)

In [17]:
CHANNELS=[2,8] #detector channels
COINCIDENCES = [2,8] #what you want the histogram to count
BIAS_CURRENTS = [2,2]
THRESHOLDS = [0.08,0.06]
HISTOGRAM_WINDOW = 50
GATE_WIDTH = 2 #coincidence window
DELAYS = [0,0]

#prepare the bias currents
for i,_ in enumerate(CHANNELS):
    set_bias(system='S', channel=CHANNELS[i], current_uA=BIAS_CURRENTS[i])
    delatch(system='S', channel=CHANNELS[i])
    
# SNSPD = det(channels = CHANNELS, coincidences=COINCIDENCES, histogram_channels=CHANNELS,
#             histogram_windows_ns=HISTOGRAM_WINDOW, 
#             user_name='Ryan', gate_width_ns=GATE_WIDTH,thresholds = THRESHOLDS, delay=DELAYS)
# print('Connected on det.phy.bris.ac.uk on channels '+str(SNSPD.__dict__['channels']))
# delatch_all(CHANNELS)


# Attenuation sweep

In [15]:
att_vals = [60,10,6.99,5.23,3.98,3.01,2.22,1.55,0.97,0.46,0]

n = 10 #number of sweeps (1, 3, 5 or 10)
t = 120 #integration time

CHANNELS = [2,8]
COINCIDENCES=[2,8]

THRESHOLDS=[0.12,0.18]
GATE_WIDTH=2 #ns
HISTOGRAM_WINDOW = 50 # ns

ch1 = 18
ch2 = 22

SNSPD = det(channels = CHANNELS, coincidences=COINCIDENCES, histogram_channels=CHANNELS,histogram_windows_ns=HISTOGRAM_WINDOW, user_name='Dan', gate_width_ns=GATE_WIDTH,thresholds = THRESHOLDS,delay=DELAYS)
print('Connected on det.phy.bris.ac.uk on channels '+str(SNSPD.__dict__['channels']))

time.sleep(1)
delatch(system='S', channel=ch1)
delatch(system='S', channel=ch2)

time.sleep(1)

if n == 1:
    dict = {'Power in [mW]' : [], 'Power on chip [mW]' : [], 'Ch 1 counts [Hz] (1)': [], 'Ch 2 counts [Hz] (1)': [], 'Coincidences [Hz] (1)': [], 'Histogram (1)': []}
elif n == 3:
    dict = {'Power in [mW]' : [], 'Power on chip [mW]' : [], 'Ch 1 counts [Hz] (1)': [], 'Ch 1 counts [Hz] (2)': [], 'Ch 1 counts [Hz] (3)': [], 'Ch 2 counts [Hz] (1)': [], 'Ch 2 counts [Hz] (2)': [], 'Ch 2 counts [Hz] (3)': [], 'Coincidences [Hz] (1)': [], 'Coincidences [Hz] (2)': [], 'Coincidences [Hz] (3)': [], 'Histogram (1)': [], 'Histogram (2)': [], 'Histogram (3)': []}
elif n == 5:
    dict = {'Power in [mW]' : [], 'Power on chip [mW]' : [], 'Ch 1 counts [Hz] (1)': [], 'Ch 1 counts [Hz] (2)': [], 'Ch 1 counts [Hz] (3)': [], 'Ch 1 counts [Hz] (4)': [], 'Ch 1 counts [Hz] (5)': [], 'Ch 2 counts [Hz] (1)': [], 'Ch 2 counts [Hz] (2)': [], 'Ch 2 counts [Hz] (3)': [], 'Ch 2 counts [Hz] (4)': [], 'Ch 2 counts [Hz] (5)': [], 'Coincidences [Hz] (1)': [], 'Coincidences [Hz] (2)': [], 'Coincidences [Hz] (3)': [], 'Coincidences [Hz] (4)': [], 'Coincidences [Hz] (5)': [], 'Histogram (1)': [], 'Histogram (2)': [], 'Histogram (3)': [], 'Histogram (4)': [], 'Histogram (5)': []}
elif n == 10:
    dict = {'Power in [mW]' : [], 'Power on chip [mW]' : [], 'Ch 1 counts [Hz] (1)': [], 'Ch 1 counts [Hz] (2)': [], 'Ch 1 counts [Hz] (3)': [], 'Ch 1 counts [Hz] (4)': [], 'Ch 1 counts [Hz] (5)': [], 'Ch 1 counts [Hz] (6)': [], 'Ch 1 counts [Hz] (7)': [], 'Ch 1 counts [Hz] (8)': [], 'Ch 1 counts [Hz] (9)': [], 'Ch 1 counts [Hz] (10)': [], 'Ch 2 counts [Hz] (1)': [], 'Ch 2 counts [Hz] (2)': [], 'Ch 2 counts [Hz] (3)': [], 'Ch 2 counts [Hz] (4)': [], 'Ch 2 counts [Hz] (5)': [], 'Ch 2 counts [Hz] (6)': [], 'Ch 2 counts [Hz] (7)': [], 'Ch 2 counts [Hz] (8)': [], 'Ch 2 counts [Hz] (9)': [], 'Ch 2 counts [Hz] (10)': [], 'Coincidences [Hz] (1)': [], 'Coincidences [Hz] (2)': [], 'Coincidences [Hz] (3)': [], 'Coincidences [Hz] (4)': [], 'Coincidences [Hz] (5)': [], 'Coincidences [Hz] (6)': [],  'Coincidences [Hz] (7)': [], 'Coincidences [Hz] (8)': [], 'Coincidences [Hz] (9)': [], 'Coincidences [Hz] (10)': [],'Histogram (1)': [], 'Histogram (2)': [], 'Histogram (3)': [], 'Histogram (4)': [], 'Histogram (5)': [], 'Histogram (6)': [], 'Histogram (7)': [], 'Histogram (8)': [], 'Histogram (9)': [], 'Histogram (10)': []}

#get power in values

coupling_loss = 28.64 #meausred coupling loss in dB

for i in att_vals:
    atten.configure(float(i), 1551)
    atten.enable()
    print('Attenuation = ' +str(i)+' dB')
    time.sleep(1)
    p_read_tap = float(pyvisa.ResourceManager().open_resource('USB0::0x1313::0x8070::P0000565::0::INSTR').query('read?'))
    p_in_dBm = 10*np.log10(1000*p_read_tap) + 15.54
    p_in_mW = 10**(p_in_dBm/10) #power before chip in mW
    p_on_chip_dBm = p_in_dBm - (coupling_loss/2)
    p_on_chip_mW = 10**(p_on_chip_dBm/10) #power in chip calculated from loss of one grating coupler
    print('Power in = ' + str(p_in_mW) + ' mW')
    print('Power on chip = ' + str(p_on_chip_mW) + ' mW')
    dict['Power in [mW]'].append(p_in_mW)
    dict['Power on chip [mW]'].append(p_on_chip_mW)
    print('-------------------------------')
    atten.disable()
    time.sleep(1)

time.sleep(1)

#run attenuation sweep
    
counter = 0
while counter < n:
    print('Measurement ' + str(counter + 1) + '/' + str(n))
    counter+=1
    for i in att_vals:
        atten.configure(float(i), 1551)
        atten.enable()
        print('Attenuation = ' +str(i)+' dB')
        print('Start: ' +str(time.ctime(time.time())))
        CHANNELS = [10,14]
        delatch(system='S', channel=ch1)
        delatch(system='S', channel=ch2)
        time.sleep(1)
        msg = SNSPD.integrate(inttime = t)
        print('Finish: ' +str(time.ctime(time.time())))
        dict['Ch 1 counts [Hz] (' +str(counter) +')'].append(msg['counts'][CHANNELS[0]-1]/t)
        dict['Ch 2 counts [Hz] (' +str(counter) +')'].append(msg['counts'][CHANNELS[1]-1]/t)
        dict['Coincidences [Hz] (' +str(counter) +')'].append(msg['coincidence'][0]/t)
        dict['Histogram (' +str(counter) +')'].append(msg['histogram_counts'])
        print('Singles Ch.1: {} \nSingles Ch.2: {}\nTotal coincidences: {}'.format(msg['counts'][CHANNELS[0]-1],msg['counts'][CHANNELS[1]-1],msg['coincidence'][0]))
        print('-------------------------------')
        atten.disable()
        time.sleep(1)
        continue

print(dict)
df = pd.DataFrame(dict)
print(df)



type : setup
waterloo_data_mode : timetag
user_name : Dan
user_platform : python3
poll_time : 1.0
tick_resolution : 1.5625e-10
active_channels : 8704
input_threshold_volts : [-0.1, 0.05, 0.04, 0.14, 0.08, 0.05, -0.06, 0.1, 0.1, 0.09, 0.1, -0.15, 0.15, 0.18, 0.03, 0.02]
channel_delay_ns : [0.0, 0.0, 9.21875, 43.90625, 0.0, 0.0, 0.0, 0.0, 20.0, 24.84375, 0.0, 20.15625, 10.0, 0.0, 9.21875, 0.0]
coincidence_channels : []
coincidence_windows_ns : []
histogram_channels : 8704
histogram_windows_ns : 50.0
slide_co_all_mask : 0
slide_co_move_mask : 0
slide_co_width : 0
locked_channels : 0
single_filter_channels : 65535
edge_inversion_channels : 0
single_filter_minimum_hits : 2
single_filter_window_ns : 10.0
connected_users : ['Dan', 'Dan', 'Dan', 'Dan']
connected_platforms : ['web', 'python3', 'python3', 'python3']
connected_active_channels : [8704, 8704, 8704, 8704]
Connected on det.phy.bris.ac.uk on channels [10, 14]
Attenuation = 60 dB
Power in = 4.1849366879904697e-05 mW
Power on chip = 2.8

Attenuation = 0 dB
Start: Wed Nov 30 20:04:27 2022
Finish: Wed Nov 30 20:06:29 2022
Singles Ch.1: 277485 
Singles Ch.2: 42648
Total coincidences: 6
-------------------------------
Measurement 4/10
Attenuation = 60 dB
Start: Wed Nov 30 20:06:30 2022
Finish: Wed Nov 30 20:08:33 2022
Singles Ch.1: 56634 
Singles Ch.2: 36510
Total coincidences: 0
-------------------------------
Attenuation = 10 dB
Start: Wed Nov 30 20:08:34 2022
Finish: Wed Nov 30 20:10:37 2022
Singles Ch.1: 76898 
Singles Ch.2: 37122
Total coincidences: 0
-------------------------------
Attenuation = 6.99 dB
Start: Wed Nov 30 20:10:38 2022
Finish: Wed Nov 30 20:12:41 2022
Singles Ch.1: 97606 
Singles Ch.2: 37456
Total coincidences: 0
-------------------------------
Attenuation = 5.23 dB
Start: Wed Nov 30 20:12:42 2022
Finish: Wed Nov 30 20:14:44 2022
Singles Ch.1: 118871 
Singles Ch.2: 37736
Total coincidences: 0
-------------------------------
Attenuation = 3.98 dB
Start: Wed Nov 30 20:14:45 2022
Finish: Wed Nov 30 20:16

Measurement 8/10
Attenuation = 60 dB
Start: Wed Nov 30 21:37:02 2022
Finish: Wed Nov 30 21:39:04 2022
Singles Ch.1: 56103 
Singles Ch.2: 36166
Total coincidences: 0
-------------------------------
Attenuation = 10 dB
Start: Wed Nov 30 21:39:05 2022
Finish: Wed Nov 30 21:41:08 2022
Singles Ch.1: 76484 
Singles Ch.2: 36088
Total coincidences: 0
-------------------------------
Attenuation = 6.99 dB
Start: Wed Nov 30 21:41:09 2022
Finish: Wed Nov 30 21:43:12 2022
Singles Ch.1: 97601 
Singles Ch.2: 37123
Total coincidences: 0
-------------------------------
Attenuation = 5.23 dB
Start: Wed Nov 30 21:43:13 2022
Finish: Wed Nov 30 21:45:15 2022
Singles Ch.1: 118757 
Singles Ch.2: 37630
Total coincidences: 1
-------------------------------
Attenuation = 3.98 dB
Start: Wed Nov 30 21:45:16 2022
Finish: Wed Nov 30 21:49:19 2022
Singles Ch.1: 141710 
Singles Ch.2: 38171
Total coincidences: 1
-------------------------------
Attenuation = 3.01 dB
Start: Wed Nov 30 21:49:20 2022
Finish: Wed Nov 30 21

    Power in [mW]  Power on chip [mW]  Ch 1 counts [Hz] (1)  \
0    4.184937e-05        2.895269e-06            472.475000   
1    6.034025e-08        4.174526e-09            633.416667   
2    6.377056e-01        4.411845e-02            806.150000   
3    9.550365e-01        6.607238e-02            976.525000   
4    1.274576e+00        8.817909e-02           1154.475000   
5    1.584745e+00        1.096375e-01           1334.608333   
6    1.900524e+00        1.314841e-01           1537.675000   
7    2.219645e+00        1.535619e-01           1718.825000   
8    2.522592e+00        1.745208e-01           1921.800000   
9    2.811623e+00        1.945168e-01           2120.058333   
10   3.058884e+00        2.116231e-01           2264.433333   

    Ch 1 counts [Hz] (2)  Ch 1 counts [Hz] (3)  Ch 1 counts [Hz] (4)  \
0             471.241667            473.233333            471.950000   
1             631.858333            633.983333            640.816667   
2             804.933333   

In [19]:
print(dict)
df = pd.DataFrame(dict)
print(df)

{'Power in [mW]': [1.756320502030327e-06, 3.7961537838601967e-07, 0.7084715571728402, 1.055808015972867, 1.408028486607271, 1.7581626527217908, 2.103673046643714, 2.4553896446673855, 2.7886448957296, 3.10627917960269, 3.370421537773329], 'Ch 1 counts (1)': [5136, 35375, 66138, 101241, 137853, 176488, 217487, 254869, 294631, 342600, 380554], 'Ch 1 counts (2)': [925, 31262, 63515, 97334, 133283, 173660, 210566, 252767, 295466, 338724, 374517], 'Ch 1 counts (3)': [1010, 30910, 63661, 97086, 132970, 172396, 211272, 251991, 299481, 341953, 380224], 'Ch 1 counts (4)': [987, 31280, 62899, 97903, 134087, 172871, 209742, 252664, 294405, 341842, 379161], 'Ch 1 counts (5)': [928, 30908, 63105, 97798, 131952], 'Ch 1 counts (6)': [], 'Ch 1 counts (7)': [], 'Ch 1 counts (8)': [], 'Ch 1 counts (9)': [], 'Ch 1 counts (10)': [], 'Ch 2 counts (1)': [5189, 22348, 40160, 60693, 81552, 103530, 128055, 149224, 171824, 200266, 222172], 'Ch 2 counts (2)': [1061, 18780, 37832, 57598, 78244, 102067, 124883, 148

ValueError: arrays must all be same length

# Wavelength sweep

In [5]:
wl_vals = np.linspace(1550.8,1551.2, 41)

n = 3 #number of sweeps (1, 3, 5 or 10)
t = 120 #integration time

CHANNELS = [10,14]
COINCIDENCES=[10,14]
DELAYS = [24.53125,0]
THRESHOLDS=[0.09,0.18]
GATE_WIDTH=2 #ns
HISTOGRAM_WINDOW = 50 # ns

ch1 = 18
ch2 = 22

SNSPD = det(channels = CHANNELS, coincidences=COINCIDENCES, histogram_channels=CHANNELS,histogram_windows_ns=HISTOGRAM_WINDOW, user_name='Dan', gate_width_ns=GATE_WIDTH,thresholds = THRESHOLDS,delay=DELAYS)
print('Connected on det.phy.bris.ac.uk on channels '+str(SNSPD.__dict__['channels']))

time.sleep(1)
delatch(system='S', channel=ch1)
delatch(system='S', channel=ch2)

time.sleep(1)

if n == 1:
    dict = {'Wavelength [nm]' : wl_vals, 'Ch 1 counts (1)': [], 'Ch 2 counts (1)': [], 'Coincidences (1)': [], 'Histogram (1)': []}
elif n == 3:
    dict = {'Wavelength [nm]' : wl_vals, 'Ch 1 counts (1)': [], 'Ch 1 counts (2)': [], 'Ch 1 counts (3)': [], 'Ch 2 counts (1)': [], 'Ch 2 counts (2)': [], 'Ch 2 counts (3)': [], 'Coincidences (1)': [], 'Coincidences (2)': [], 'Coincidences (3)': [], 'Histogram (1)': [], 'Histogram (2)': [], 'Histogram (3)': []}
elif n == 5:
    dict = {'Wavelength [nm]' : wl_vals, 'Ch 1 counts (1)': [], 'Ch 1 counts (2)': [], 'Ch 1 counts (3)': [], 'Ch 1 counts (4)': [], 'Ch 1 counts (5)': [], 'Ch 2 counts (1)': [], 'Ch 2 counts (2)': [], 'Ch 2 counts (3)': [], 'Ch 2 counts (4)': [], 'Ch 2 counts (5)': [], 'Coincidences (1)': [], 'Coincidences (2)': [], 'Coincidences (3)': [], 'Coincidences (4)': [], 'Coincidences (5)': [], 'Histogram (1)': [], 'Histogram (2)': [], 'Histogram (3)': [], 'Histogram (4)': [], 'Histogram (5)': []}
elif n == 10:
    dict = {'Wavelength [nm]' : wl_vals, 'Ch 1 counts (1)': [], 'Ch 1 counts (2)': [], 'Ch 1 counts (3)': [], 'Ch 1 counts (4)': [], 'Ch 1 counts (5)': [], 'Ch 1 counts (6)': [], 'Ch 1 counts (7)': [], 'Ch 1 counts (8)': [], 'Ch 1 counts (9)': [], 'Ch 1 counts (10)': [], 'Ch 2 counts (1)': [], 'Ch 2 counts (2)': [], 'Ch 2 counts (3)': [], 'Ch 2 counts (4)': [], 'Ch 2 counts (5)': [], 'Ch 2 counts (6)': [], 'Ch 2 counts (7)': [], 'Ch 2 counts (8)': [], 'Ch 2 counts (9)': [], 'Ch 2 counts (10)': [], 'Coincidences (1)': [], 'Coincidences (2)': [], 'Coincidences (3)': [], 'Coincidences (4)': [], 'Coincidences (5)': [], 'Coincidences (6)': [],  'Coincidences (7)': [], 'Coincidences (8)': [], 'Coincidences (9)': [], 'Coincidences (10)': [],'Histogram (1)': [], 'Histogram (2)': [], 'Histogram (3)': [], 'Histogram (4)': [], 'Histogram (5)': [], 'Histogram (6)': [], 'Histogram (7)': [], 'Histogram (8)': [], 'Histogram (9)': [], 'Histogram (10)': []}


time.sleep(1)

#run wavelength sweep
    
counter = 0
while counter < n:
    print('Measurement ' + str(counter + 1) + '/' + str(n))
    counter+=1
    for i in wl_vals:
        print('Start: ' +str(time.ctime(time.time())))
        try: 
            pyvisa.ResourceManager().open_resource('GPIB::8::INSTR').query('L='+str(i))
        except pyvisa.VisaIOError:
            CHANNELS = [10,14]
            delatch(system='S', channel=ch1)
            delatch(system='S', channel=ch2)
            time.sleep(1)
            print('Integrating...')
            msg = SNSPD.integrate(inttime = t)
            print('Finish: ' +str(time.ctime(time.time())))
            dict['Ch 1 counts (' +str(counter) +')'].append(msg['counts'][CHANNELS[0]-1])
            dict['Ch 2 counts (' +str(counter) +')'].append(msg['counts'][CHANNELS[1]-1])
            dict['Coincidences (' +str(counter) +')'].append(msg['coincidence'][0])
            dict['Histogram (' +str(counter) +')'].append(msg['histogram_counts'])
            print('Singles Ch.1: {} \nSingles Ch.2: {}\nTotal coincidences: {}'.format(msg['counts'][CHANNELS[0]-1],msg['counts'][CHANNELS[1]-1],msg['coincidence'][0]))
            print('-------------------------------')
            time.sleep(1)
            continue

print(dict)
df = pd.DataFrame(dict)
print(df)


type : setup
waterloo_data_mode : timetag
user_name : Dan
user_platform : python3
poll_time : 1.0
tick_resolution : 1.5625e-10
active_channels : 8704
input_threshold_volts : [-0.1, 0.05, 0.04, 0.14, 0.08, 0.05, -0.06, 0.1, 0.1, 0.09, 0.1, -0.15, 0.15, 0.18, 0.03, 0.02]
channel_delay_ns : [0.0, 0.0, 9.21875, 43.90625, 0.0, 0.0, 0.0, 0.0, 20.0, 24.53125, 0.0, 20.15625, 10.0, 0.0, 9.21875, 0.0]
coincidence_channels : []
coincidence_windows_ns : []
histogram_channels : 8704
histogram_windows_ns : 50.0
slide_co_all_mask : 0
slide_co_move_mask : 0
slide_co_width : 0
locked_channels : 0
single_filter_channels : 65535
edge_inversion_channels : 0
single_filter_minimum_hits : 2
single_filter_window_ns : 10.0
connected_users : ['Dan', 'Dan']
connected_platforms : ['web', 'python3']
connected_active_channels : [8704, 8704]
Connected on det.phy.bris.ac.uk on channels [10, 14]
Measurement 1/3
Start: Mon Nov 28 18:55:58 2022
Integrating...
Finish: Mon Nov 28 19:12:03 2022
Singles Ch.1: 66371 
Singles

Start: Mon Nov 28 20:39:21 2022
Integrating...
Finish: Mon Nov 28 20:41:27 2022
Singles Ch.1: 267996 
Singles Ch.2: 148886
Total coincidences: 0
-------------------------------
Start: Mon Nov 28 20:41:28 2022
Integrating...
Finish: Mon Nov 28 20:43:35 2022
Singles Ch.1: 263483 
Singles Ch.2: 147615
Total coincidences: 1
-------------------------------
Start: Mon Nov 28 20:43:36 2022
Integrating...
Finish: Mon Nov 28 20:45:43 2022
Singles Ch.1: 261239 
Singles Ch.2: 146246
Total coincidences: 3
-------------------------------
Start: Mon Nov 28 20:45:44 2022
Integrating...
Finish: Mon Nov 28 20:47:50 2022
Singles Ch.1: 255591 
Singles Ch.2: 143698
Total coincidences: 1
-------------------------------
Start: Mon Nov 28 20:47:51 2022
Integrating...
Finish: Mon Nov 28 20:49:58 2022
Singles Ch.1: 251980 
Singles Ch.2: 142063
Total coincidences: 1
-------------------------------
Start: Mon Nov 28 20:49:59 2022
Integrating...
Finish: Mon Nov 28 20:52:05 2022
Singles Ch.1: 246269 
Singles Ch.2:

Start: Mon Nov 28 22:19:26 2022
Integrating...
Finish: Mon Nov 28 22:21:33 2022
Singles Ch.1: 217373 
Singles Ch.2: 122315
Total coincidences: 1
-------------------------------
Start: Mon Nov 28 22:21:34 2022
Integrating...
Finish: Mon Nov 28 22:23:40 2022
Singles Ch.1: 222520 
Singles Ch.2: 125461
Total coincidences: 1
-------------------------------
Start: Mon Nov 28 22:23:41 2022
Integrating...
Finish: Mon Nov 28 22:25:47 2022
Singles Ch.1: 232461 
Singles Ch.2: 130306
Total coincidences: 0
-------------------------------
Start: Mon Nov 28 22:25:48 2022
Integrating...
Finish: Mon Nov 28 22:27:55 2022
Singles Ch.1: 243056 
Singles Ch.2: 136016
Total coincidences: 0
-------------------------------
Start: Mon Nov 28 22:27:56 2022
Integrating...
Finish: Mon Nov 28 22:30:04 2022
Singles Ch.1: 249097 
Singles Ch.2: 139701
Total coincidences: 0
-------------------------------
Start: Mon Nov 28 22:30:05 2022
Integrating...
Finish: Mon Nov 28 22:32:11 2022
Singles Ch.1: 253692 
Singles Ch.2:

Start: Mon Nov 28 23:59:32 2022
Integrating...
Finish: Tue Nov 29 00:01:39 2022
Singles Ch.1: 152461 
Singles Ch.2: 88175
Total coincidences: 1
-------------------------------
Start: Tue Nov 29 00:01:40 2022
Integrating...
Finish: Tue Nov 29 00:03:46 2022
Singles Ch.1: 157018 
Singles Ch.2: 90827
Total coincidences: 0
-------------------------------
Start: Tue Nov 29 00:03:47 2022
Integrating...
Finish: Tue Nov 29 00:05:54 2022
Singles Ch.1: 161463 
Singles Ch.2: 93173
Total coincidences: 0
-------------------------------
Start: Tue Nov 29 00:05:55 2022
Integrating...
Finish: Tue Nov 29 00:08:02 2022
Singles Ch.1: 167017 
Singles Ch.2: 96944
Total coincidences: 0
-------------------------------
Start: Tue Nov 29 00:08:03 2022
Integrating...
Finish: Tue Nov 29 00:10:09 2022
Singles Ch.1: 173042 
Singles Ch.2: 99812
Total coincidences: 1
-------------------------------
Start: Tue Nov 29 00:10:10 2022
Integrating...
Finish: Tue Nov 29 00:12:17 2022
Singles Ch.1: 177742 
Singles Ch.2: 1022

ConnectionError: HTTPConnectionPool(host='det.phy.bris.ac.uk', port=80): Max retries exceeded with url: /bias2.php?cmd=delatch&system=S&channel=18 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000000054B5508>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))